## Explore PubAG API

In [1]:
import requests

In [10]:
API_KEY = "jGgEv0rYItqngfgOd9eF9HN5GoCDRxTzB64slFdL"

In [30]:
response = requests.get("https://api.nal.usda.gov/pubag/rest/search/?query=precision+agriculture&per_page=100&api_key={}".format(API_KEY)).json()

In [34]:
totalpages = response["request"]["totalPages"]
results = response["resultList"]

abstracts = []
for i in range(totalpages):
    abstracts.append(results[i]["abstract"]) 